# Sort 2D class averages using common lines

In [ ]:
import time
import mrcfile
import numpy as np
import pandas as pd
import skimage as ski
import itertools as it
import multiprocessing 
from igraph import Graph
from scipy import ndimage as ndi
from scipy import spatial, signal, stats

### Inputs

In [ ]:
#choose metric for comparing 1D projections (Euclidean, L1, cross-correlation, cosine)
metric = 'L1'

#whether or not to zscore normalize 1D projections before scoring (not recommended)
normalize = False

#path to mrcs file of 2D class averages
mrc_input = 'path/to/input/mixture_2D.mrcs

#number of CPUs to use
num_workers = 1

#pixel size of 2D class averages in A/pixel
pixel_size = 4

#number of edges for each node in the graph
neighbors = 5

#community detection algorithm to use (betweenness, walktrap)
community_detection = 'betweenness''

#path for output files
outpath = 'path/to/output/'

#name for output file
description = 'mixture'

#path to star file for particles in 2D class averages
star_input = 'path/to/matching/mixture_particles.star'

### read in 2D projections

In [ ]:
class Projection:
    """for 1D projection vectors"""
    def __init__(self, 
                 class_avg,
                 angle,
                 vector): 

        self.class_avg = class_avg
        self.angle = angle
        self.vector = vector
    
    def size(self):
        l = len(self.vector)
        return(l)
    
    @property
    def dimension(self):
        return self.vector.ndim
    
def get_1D_projections(mrcs, norm):
    """
    creates dictionary of 1D projections, key is [class avg number, projection angle] 
    """
    projection_2D = {}
    
    projection_1D = {}
    
    angles = np.arange(0, 360, 5)
    
    with mrcfile.open(mrc_input) as mrc:
        for i, data in enumerate(mrc.data):
            projection_2D[i] = data.astype('float64')
            
    for k, avg in projection_2D.items():
        projection_2D[k] = extract_class_avg(avg)
    
    for k, avg in projection_2D.items():
        for a in angles:
            #to mimick original version, don't resize or trim avgs
            proj_1D = ski.transform.rotate(avg, a, resize=True).sum(axis=0)
            trim_1D = np.trim_zeros(proj_1D, trim='fb')
            projection_1D[(k, a)] = Projection(class_avg = k,
                                                   angle = a,
                                                   vector = trim_1D)

    if norm:
        for k, v in projection_1D.items():
            projection_1D[k].vector = stats.zscore(projection_1D[k].vector)
    
    return projection_2D, projection_1D  

def extract_class_avg(avg):
    """
    keep positive values from image normalization
    remove extra densities in class average
    fit in minimal bounding box
    """
    #TODO: try different region extraction or masking
    pos_img = avg
    pos_img[pos_img < 0] = 0
    
    #dilate by pixel size to connect neighbor regions
    extra = 3 #Angrstroms to dilate (set minimum of 3A)
    extend = int(np.ceil((pixel_size/extra)**-1))

    struct = np.ones((extend, extend), dtype=bool)
    dilate = ndi.binary_dilation(input=pos_img, structure=struct)

    labeled = ski.measure.label(dilate, connectivity=2, background=False)

    #select a single region from each 2D class average
    rprops = ski.measure.regionprops(labeled, cache=False)
    bbox = [r.bbox for r in rprops]

    if len(bbox) == 1:
        #use only region in the image
        selected = 1

    elif len(bbox) > 1:
        img_x_center = len(avg)/2
        img_y_center = len(avg[:,0])/2
        img_center = (img_x_center, img_y_center)
        #for use in distance calculation
        x1 = np.array(img_center)

        box_range = {}
        for i, box in enumerate(bbox):
            width_coord = list(range(box[1], box[3]+1))
            length_coord = list(range(box[0], box[2]+1))
            box_range[i+1] = [width_coord, length_coord]

        box_centers = {}
        for i, box in enumerate(bbox):
            y_max, y_min = box[0], box[2]
            x_max, x_min = box[1], box[3]
            center_xy = (((x_max+x_min)/2, (y_max+y_min)/2))
            #i+1 because 0 is the background region
            box_centers[i+1] = center_xy

        selected = 'none'

        for region, bound in box_range.items():
            #first check if there is a region in the center
            if img_x_center in bound[0] and img_y_center in bound[1]:
                #use center region
                selected = region

        if selected == 'none':
            #find box closest to the center    
            distance = {}
            for region, center in box_centers.items():
                x2 = np.array(center)
                distance[region] = spatial.distance.euclidean(x1, x2)  
            region = min(distance, key=distance.get) 
            #use region closest to center
            selected = region

    selected_region = (labeled == selected)

    properties = ski.measure.regionprops(selected_region.astype('int'))
    bbox = properties[0].bbox

    y_min, y_max = bbox[0], bbox[2]
    x_min, x_max = bbox[1], bbox[3]

    #keep only true pixels in bounding box
    true_region = np.empty(avg.shape)

    for i, row in enumerate(avg):
        for j, pixel in enumerate(row):
            if selected_region[(i,j)] == True:
                true_region[(i,j)] = pos_img[(i, j)]
            else:
                true_region[(i,j)] = 0

    new_region = true_region[y_min:y_max, x_min:x_max]

    return new_region

In [ ]:
projection_2D, projection_1D = get_1D_projections(mrcs=mrc_input, norm=normalize)
num_class_avg = len(projection_2D)

n = len(projection_1D)
print("number of 2D class averages: {}".format(num_class_avg))
print("number of 1D projection vectors: {}".format(n))
print("total number of pairwise scores: {}".format(int(n*(n-1)/2)))

#remove pairs from same 2D class average
all_pairs = list(it.combinations(projection_1D.values(), 2))
projection_pairs = [pair for pair in all_pairs if pair[0].class_avg != pair[1].class_avg]

### sliding vector comparisons

In [ ]:
if metric == 'Euclidean':
    def pairwise_score(a, b):
        score = spatial.distance.euclidean(a.vector, b.vector)
        return score
        
elif metric == 'L1':
    def pairwise_score(a, b):
        score = sum(abs(a.vector - b.vector))
        return score
    
elif metric == 'cosine':
    def pairwise_score(a, b):
        score = spatial.distance.cosine(a.vector - b.vector)
        return score   

elif metric == 'cross-correlation':
    def pairwise_score(a, b):
        score = signal.correlate(a.vector, b.vector, mode='valid')
        score_max = np.amax(score)
        return score_max

def slide_score(a, b):
    """
    a, b are instances of the Projection class
    finds minimum pairwise score for all translations
    """
    lp1 = a.size()
    lp2 = b.size()
    
    dol = abs(lp1 - lp2)
    
    if dol == 0:
        score = pairwise_score(a, b)

    else:
        projection_shifts = []
        
        if lp1 > lp2:    
            static = a
            shift = b
        else:
            static = b
            shift = a
            
        for i in range(0, dol+1):
            v = np.pad(shift.vector, pad_width=(i, dol-i), mode='constant')
            projection_shifts.append(Projection(class_avg = str(shift.class_avg)+'_'+str(i),
                                                angle = shift.angle,
                                                vector = v))

        scores = []
        
        for shifted in projection_shifts:
            scores.append(pairwise_score(static, shifted))
        score = min(scores)
        
    return score

def wrapper_function(pair, func):
    """
    - pair is tuple of instances from Projection class
    - func is how to pairwise score vectors defined from user
    """
    score = func(pair[0], pair[1])
    
    return [pair[0].class_avg, 
            pair[0].angle, 
            pair[1].class_avg, 
            pair[1].angle,
            score]

def multiproc_pairwise():
    with multiprocessing.Pool(num_workers) as pool:
        if metric != 'cross-correlation':
            pair_scores = pool.starmap(wrapper_function, 
                                       it.product(projection_pairs, [slide_score]))
        else:
            pair_scores = pool.starmap(wrapper_function, 
                                       it.product(projection_pairs, [pairwise_score]))
    return pair_scores

In [ ]:
if __name__ == "__main__":
    starttime = time.time()
    scores = multiproc_pairwise()
    print('That took: {} minutes'.format((time.time() - starttime)/60))
    
pairs = list(it.combinations(list(range(num_class_avg)), 2))

score_matrix = {pair: [] for pair in pairs}

for s in scores:
    #format is [projection_1, projection_2] = [(angle_1, angle_2, score)]
    score_matrix[s[0], s[2]].append((s[1], s[3], s[4]))

### find best pairwise scores between 2D classes

In [ ]:
def optimum_scores(score_matrix):
    """
    best pairwise score for all 1D projections
    """
    optimum_pairwise_scores = {}
    
    complete_score_matrix = {}

    #cross-correlation optimum is max, other metrics are min
    if metric != 'cross-correlation': 
        for pair, scores in score_matrix.items():
            optimum_pairwise_scores[pair] = min(score_matrix[pair], key = lambda x: x[2])
    else:
        for pair, scores in score_matrix.items():
            optimum_pairwise_scores[pair] = max(score_matrix[pair], key = lambda x: x[2])

    #complete the matrix
    for pair, value in optimum_pairwise_scores.items():
        if pair[0] == pair[1]:
            next
        else:
            complete_score_matrix[pair] = value
            #now flip pair
            score = value[2]
            angle_1 = value[0]
            angle_2 = value[1]
            complete_score_matrix[(pair[1], pair[0])] = (angle_2, angle_1, score)
        
    return complete_score_matrix

In [ ]:
complete_scores = optimum_scores(score_matrix)

### nearest neighbors for each 2D class

In [ ]:
def nearest_neighbors(complete_scores):
    """
    group k best scores for each class average to construct graph 
    """
    order_scores = {avg: [] for avg in range(num_class_avg)}
    
    projection_knn = {}

    #reorganize score matrix to sort for nearest neighbors
    #projection_knn[projection_1] = [projection_2, angle_1, angle_2, score]
    for pair, values in complete_scores.items():
        p1 = pair[0]
        p2 = pair[1]
        a1 = values[0]
        a2 = values[1]
        s = values[2]
        
        #option: convert distance to similarity
        #if metric != 'cross-correlation':
        #    s = 1/(1 + values[2])
        #else:
        #    s = values[2]
            
        c = [p2, a1, a2, s]
        order_scores[p1].append(c)
        
    #zscore scores by each projection, use as edgeweight in graph
    for projection, scores in order_scores.items():
        all_scores = []
        for v in scores:
            all_scores.append(v[3])
        
        u = np.mean(all_scores)
        s = np.std(all_scores)
    
        for v in scores:
            zscore = (v[3] - u)/s
            v[3] = zscore

    for avg, scores in order_scores.items():
        if metric != 'cross-correlation':
            sort = sorted(scores, reverse=False, key=lambda x: x[3])[:neighbors]
        else:
            sort = sorted(scores, reverse=True, key=lambda x: x[3])[:neighbors]
        projection_knn[avg] = sort
                
    return projection_knn

In [ ]:
projection_knn = nearest_neighbors(complete_scores=complete_scores)

### use graph to cluster 2D classes

In [ ]:
def detect_communities(projection_knn):
    """
    cluster graph using walktrap of edge betweeness
    """
    flat = []
    
    clusters = {}

    for projection, knn in projection_knn.items():
        for n in knn:
            #(proj_1, proj_2, score)
            #abs score to correct for distance vs similarity
            flat.append((projection, n[0], abs(n[3])))

    g = Graph.TupleList(flat, weights=True)

    if community_detection == 'walktrap':
        wt = Graph.community_walktrap(g, weights='weight', steps=4)
        cluster_dendrogram = wt.as_clustering()
    elif community_detection == 'betweenness':
        ebs = Graph.community_edge_betweenness(g, weights='weight', directed=True)
        cluster_dendrogram = ebs.as_clustering()

    for community, projection in enumerate(cluster_dendrogram.subgraphs()):
        clusters[community] = projection.vs['name']

    #convert node IDs back to ints
    for cluster, nodes in clusters.items():
        clusters[cluster] = [int(node) for node in nodes]
        
    return clusters


def remove_outliers(clusters):
    """
    use median absolute deviation of summed 2D projections to remove outliers
    inspect outliers for further processing
    """
    pixel_sums = {}
    
    outliers = []

    for cluster, nodes in clusters.items():
        pixel_sums[cluster] = []
        for node in nodes:
            pixel_sums[cluster].append(sum(sum(projection_2D[node])))

    for cluster, psums in pixel_sums.items():
        med = np.median(psums)
        m_psums = [abs(x - med) for x in psums]
        mad = np.median(m_psums)

        for i, proj in enumerate(psums):
            #Boris Iglewicz and David Hoaglin (1993)
            z = 0.6745*(proj - med)/mad
            if abs(z) > 3.5:
                outliers.append((cluster, clusters[cluster][i]))

    clusters["outliers"] = [o[1] for o in outliers]
    
    for outlier in outliers:
        cluster = outlier[0]
        node = outlier[1]
        clusters[cluster].remove(node)
        print('projection node {0} was removed from cluster {1} as an outlier'.format(node, cluster))

In [ ]:
clusters = detect_communities(projection_knn=projection_knn)

remove_outliers(clusters=clusters)

### write score outputs

In [ ]:
def write_scores():
    """
    tab separted text file of nearest neighbors 
    and complete score matrix
    neighbors file can be input to cytoscape
    """
    #tab separated: projection_1, angle_1, projection_2, angle_2, score    
    with open(outpath+'/neighbors_{0}.txt'.format(description), 'w') as f:
        f.write('projection_1' + '\t' +'angle_1' + '\t'\
                + 'projection_2' + '\t' + 'angle_2' + '\t' + 'edge_score' + '\n')
        for projection, neighbors in projection_knn.items():
            for n in neighbors:
                f.write(str(projection)+'\t'+str(n[1])+'\t'+str(n[0])+'\t'+str(n[2])+'\t'+str(n[3])+'\n')

    with open(outpath+'/complete_scores_{0}.txt'.format(description), 'w') as f:
        f.write('projection_1' + '\t' + 'angle_1' + '\t' +'projection_2' +\
                '\t' + 'angle_2' + '\t' + 'score' + '\n')
        for p, v in complete_scores.items():
            f.write(str(p[0])+'\t'+str(v[0])+'\t'+str(p[1])+'\t'+str(v[1])+'\t'+str(v[2])+'\n')

In [ ]:
write_scores()

### split star file by clusters

In [ ]:
"""
functions to parse star file adapted from Tristan Bepler
https://github.com/tbepler/topaz
https://www.nature.com/articles/s41592-019-0575-8
"""

def parse_star(f):
    return parse(f)

def parse(f):
    lines = f.readlines()
    for i in range(len(lines)):
        line = lines[i]
        if line.startswith('data_'): 
            return parse_star_body(lines[i+1:])
        
def parse_star_body(lines):
    #data_images line has been read, next is loop
    for i in range(len(lines)):
        if lines[i].startswith('loop_'):
            lines = lines[i+1:]
            break
    header,lines = parse_star_loop(lines)
    #parse the body
    content = []
    for i in range(len(lines)):
        line = lines[i].strip()
        if line.startswith('data'): # done with image block
            break
        if line.startswith('#') or line.startswith(';'): # comment lines
            continue
        if line != '':
            tokens = line.split()
            content.append(tokens)
            
    table = pd.DataFrame(content, columns=header)
    
    return table      
    
def parse_star_loop(lines):
    columns = []
    for i in range(len(lines)):
        line = lines[i].strip()
        if not line.startswith('_'):
            break
        name = line[1:]
        #strip trailing comments from name
        loc = name.find('#')
        if loc >= 0:
            name = name[:loc]
        #strip 'rln' prefix
        if name.startswith('rln'):
            name = name[3:]
        name = name.strip()
        
        columns.append(name)
        
    return columns, lines[i:]

def write_star_files():
    """
    split star file into new star files based on clusters
    """
    with open(star_input, 'r') as f:
        table = parse_star(f)

    cluster_star = {}

    for cluster, nodes in clusters.items():
        #convert to str to match df
        #add 1 to match RELION indexing
        avgs = [str(node+1) for node in nodes]
        subset = table[table['ClassNumber'].isin(avgs)]
        cluster_star[cluster] = subset

    for cluster, table in cluster_star.items():
        with open(outpath+'/{0}_cluster_{1}.star'.format(description, cluster), 'w') as f:
            #write the star file
            print('data_', file=f)
            print('loop_', file=f)
            for i, name in enumerate(table.columns):
                print('_rln' + name + ' #' + str(i+1), file=f)
            table.to_csv(f, sep='\t', index=False, header=False)
            
    with open(outpath+'/{0}_clusters.txt'.format(description), 'w') as f:
        for cluster, averages in clusters.items():
            f.write(str(cluster) + '\t' + str(averages) + '\n')

In [ ]:
write_star_files()